In [1]:
import sys

sys.path.append("../")
from calculus_path_mod.term_engine import *
from calculus_path_mod.reduction_strategy import *

Lambda = Abstraction
App = Application

# <span style="color: #A1F99C">TEST PROCEDURE</span>

In [2]:
rs = RandomInnerStrategy(prob_norm="sum")


def test_procedure(term, reduction_strategy=rs, print_steps=True,
                   steps_lim=60, assert_result=""):
    term = term._update_bound_vars()
    n_steps = 0
    is_term_normalized = True
    if print_steps:
        print(f"{n_steps}: {term}")
    while term.redexes:
        term = term._beta_conversion(reduction_strategy)
        term = term._update_bound_vars()
        n_steps += 1
        if print_steps:
            print(f"{n_steps}: {term}")
        if n_steps > steps_lim:
            is_term_normalized = False
            break

    print(f"\n\nreduction steps: {n_steps}")
    print(f"norm term:       {term}" if is_term_normalized else f"NOT norm term: {term}")

# <span style="color: #A1F99C">USEFUL TERMS</span>

In [3]:
def ite_term():
    x, y, c = Var(), Var(), Var()
    x_, y_, c_ = Atom(x), Atom(y), Atom(c)
    return Lambda(c, Lambda(x, Lambda(y, multi_app_term(c_, x_, y_))))


def true_term():
    x, y = Var(), Var()
    x_ = Atom(x)
    return Lambda(x, Lambda(y, x_))


def num_zero_term():
    s, z = Var(), Var()
    z_ = Atom(z)
    return Lambda(s, Lambda(z, z_))


def num_term(n: int):
    if n <= 0:
        return num_zero_term()
    s, z = Var(), Var()
    s_, z_ = Atom(s), Atom(z)
    core_term = App(s_, z_)
    for _ in range(n - 1):
        core_term = App(s_, core_term)
    return Lambda(s, Lambda(z, core_term))


def multi_app_term(term_0: Term, term_1: Term, *terms: Term):
    res_app_term = App(term_0, term_1)
    for term in terms:
        res_app_term = App(res_app_term, term)
    return res_app_term


def false_term():
    x, y = Var(), Var()
    y_ = Atom(y)
    return Lambda(x, Lambda(y, y_))


def pair_term():
    x, y, p = Var(), Var(), Var()
    x_, y_, p_ = Atom(x), Atom(y), Atom(p)
    return Lambda(x, Lambda(y, Lambda(p, multi_app_term(p_, x_, y_))))


def first_term():
    p = Var()
    p_ = Atom(p)
    return Lambda(p, App(p_, true_term()))


def second_term():
    p = Var()
    p_ = Atom(p)
    return Lambda(p, App(p_, false_term()))


def succ_term():
    x, y, n = Var(), Var(), Var()
    x_, y_, n_ = Atom(x), Atom(y), Atom(n)
    return Lambda(n, Lambda(x, Lambda(y, App(x_, multi_app_term(n_, x_, y_)))))


def sinc_term():
    p = Var()
    p_ = Atom(p)
    return Lambda(
        p,
        multi_app_term(
            pair_term(),
            App(second_term(), p_),
            App(succ_term(), App(second_term(), p_)),
        ),
    )


def pred_term():
    n, f, x, g, h, u = Var(), Var(), Var(), Var(), Var(), Var()
    n_, f_, x_ = Atom(n), Atom(f), Atom(x)
    g_, h_, u_ = Atom(g), Atom(h), Atom(u)

    return Lambda(n, Lambda(f, Lambda(x, multi_app_term(
        n_,
        Lambda(g, Lambda(h, App(h_, App(g_, f_)))),
        Lambda(u, x_),
        Lambda(u, u_)
    ))))


def plus_term():
    n, m = Var(), Var()
    n_, m_ = Atom(n), Atom(m)
    return Lambda(m, Lambda(n, multi_app_term(n_, succ_term(), m_)))


def subtract_term():
    n, m = Var(), Var()
    n_, m_ = Atom(n), Atom(m)
    return Lambda(n, Lambda(m, multi_app_term(m_, pred_term(), n_)))


def iszero_term():
    x, n = Var(), Var()
    n_ = Atom(n)
    return Lambda(n, App(App(n_, Lambda(x, false_term())), true_term()))


def leq_term():
    n, m = Var(), Var()
    n_, m_ = Atom(n), Atom(m)
    return Lambda(
        n, Lambda(m, App(iszero_term(), multi_app_term(subtract_term(), n_, m_)))
    )


def and_term():
    a, b = Var(), Var()
    a_, b_ = Atom(a), Atom(b)
    return Lambda(a, Lambda(b, multi_app_term(ite_term(), a_, b_, a_)))


def eq_term():
    n, m = Var(), Var()
    n_, m_ = Atom(n), Atom(m)
    return Lambda(
        n,
        Lambda(
            m,
            multi_app_term(
                and_term(),
                multi_app_term(leq_term(), n_, m_),
                multi_app_term(leq_term(), m_, n_),
            ),
        ),
    )

# <span style="color: #A1F99C">1.</span>
<span style="color:#bb91f3">(λx. (λy.y)) ((λx. (xx)) (λx. (xx)))</span> == <span style="color:#f1bd47">I</span> == <span style="color:#f1bd47">λx. x</span>
<span style="color:#c9fafa">This term reduce in one step by LO, RO strategy
And reduce in the same terms with other strategy.</span>

In [4]:
def term_test_1():
    x, y = Var(), Var()
    x_, y_ = Atom(x), Atom(y)

    return App(
        Lambda(x, Lambda(y, y_)),
        App(
            Lambda(x, App(x_, x_)),
            Lambda(x, App(x_, x_))
        )
    )


test_procedure(term_test_1())

0: ((λ#2. (λ#3. #3)) ((λ#4. (#4 #4)) (λ#5. (#5 #5))))
1: ((λ#6. (λ#7. #7)) ((λ#8. (#8 #8)) (λ#9. (#9 #9))))
2: ((λ#10. (λ#11. #11)) ((λ#12. (#12 #12)) (λ#13. (#13 #13))))
3: ((λ#14. (λ#15. #15)) ((λ#16. (#16 #16)) (λ#17. (#17 #17))))
4: (λ#18. #18)


reduction steps: 4
norm term:       (λ#18. #18)


# <span style="color: #A1F99C">2.</span>
<span style="color:#bb91f3">(ITE TRUE 3 1)</span> == <span style="color:#f1bd47>">3</span> == <span style="color:#f1bd47">λx. λy. (x (x (x y)))</span>

In [5]:
def term_test_2():
    return multi_app_term(ite_term(), true_term(), num_term(3), num_term(1))


test_procedure(term_test_2())

0: ((((λ#28. (λ#29. (λ#30. ((#28 #29) #30)))) (λ#31. (λ#32. #31))) (λ#33. (λ#34. (#33 (#33 (#33 #34)))))) (λ#35. (λ#36. (#35 #36))))
1: (((λ#37. (λ#38. (((λ#39. (λ#40. #39)) #37) #38))) (λ#41. (λ#42. (#41 (#41 (#41 #42)))))) (λ#43. (λ#44. (#43 #44))))
2: ((λ#45. (((λ#46. (λ#47. #46)) (λ#48. (λ#49. (#48 (#48 (#48 #49)))))) #45)) (λ#50. (λ#51. (#50 #51))))
3: ((λ#52. ((λ#53. (λ#54. (λ#55. (#54 (#54 (#54 #55)))))) #52)) (λ#56. (λ#57. (#56 #57))))
4: ((λ#58. (λ#59. (λ#60. (#59 (#59 (#59 #60)))))) (λ#61. (λ#62. (#61 #62))))
5: (λ#63. (λ#64. (#63 (#63 (#63 #64)))))


reduction steps: 5
norm term:       (λ#63. (λ#64. (#63 (#63 (#63 #64)))))


# <span style="color: #A1F99C">3.</span>
<span style="color:#bb91f3">(ITE FALSE 0 4)</span> == <span style="color:#f1bd47">4</span> == <span style="color:#f1bd47">λx. λy. (x (x (x (x y))))</span>

In [6]:
def term_test_3():
    return multi_app_term(ite_term(), false_term(), num_term(0), num_term(4))


test_procedure(term_test_3())

0: ((((λ#74. (λ#75. (λ#76. ((#74 #75) #76)))) (λ#77. (λ#78. #78))) (λ#79. (λ#80. #80))) (λ#81. (λ#82. (#81 (#81 (#81 (#81 #82)))))))
1: (((λ#83. (λ#84. (((λ#85. (λ#86. #86)) #83) #84))) (λ#87. (λ#88. #88))) (λ#89. (λ#90. (#89 (#89 (#89 (#89 #90)))))))
2: (((λ#91. (λ#92. ((λ#93. #93) #92))) (λ#94. (λ#95. #95))) (λ#96. (λ#97. (#96 (#96 (#96 (#96 #97)))))))
3: ((λ#98. ((λ#99. #99) #98)) (λ#100. (λ#101. (#100 (#100 (#100 (#100 #101)))))))
4: ((λ#102. #102) (λ#103. (λ#104. (#103 (#103 (#103 (#103 #104)))))))
5: (λ#105. (λ#106. (#105 (#105 (#105 (#105 #106))))))


reduction steps: 5
norm term:       (λ#105. (λ#106. (#105 (#105 (#105 (#105 #106))))))


# <span style="color: #A1F99C">4.</span>
<span style="color:#bb91f3">(PRED 0)</span> == <span style="color:#f1bd47">0</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [7]:
def term_test_4():
    return multi_app_term(pred_term(), num_term(0))


test_procedure(term_test_4())

0: ((λ#115. (λ#116. (λ#117. (((#115 (λ#118. (λ#119. (#119 (#118 #116))))) (λ#120. #117)) (λ#121. #121))))) (λ#122. (λ#123. #123)))
1: (λ#124. (λ#125. ((((λ#126. (λ#127. #127)) (λ#128. (λ#129. (#129 (#128 #124))))) (λ#130. #125)) (λ#131. #131))))
2: (λ#132. (λ#133. (((λ#134. #134) (λ#135. #133)) (λ#136. #136))))
3: (λ#137. (λ#138. ((λ#139. #138) (λ#140. #140))))
4: (λ#141. (λ#142. #142))


reduction steps: 4
norm term:       (λ#141. (λ#142. #142))


# <span style="color: #A1F99C">5.</span>
<span style="color:#bb91f3">(PRED 5)</span> == <span style="color:#f1bd47">4</span> == <span style="color:#f1bd47">λx. λy. (x (x (x (x y))))</span>

In [8]:
def term_test_5():
    return multi_app_term(pred_term(), num_term(5))


test_procedure(term_test_5())

0: ((λ#151. (λ#152. (λ#153. (((#151 (λ#154. (λ#155. (#155 (#154 #152))))) (λ#156. #153)) (λ#157. #157))))) (λ#158. (λ#159. (#158 (#158 (#158 (#158 (#158 #159))))))))
1: (λ#160. (λ#161. ((((λ#162. (λ#163. (#162 (#162 (#162 (#162 (#162 #163))))))) (λ#164. (λ#165. (#165 (#164 #160))))) (λ#166. #161)) (λ#167. #167))))
2: (λ#168. (λ#169. (((λ#170. ((λ#171. (λ#172. (#172 (#171 #168)))) ((λ#173. (λ#174. (#174 (#173 #168)))) ((λ#175. (λ#176. (#176 (#175 #168)))) ((λ#177. (λ#178. (#178 (#177 #168)))) ((λ#179. (λ#180. (#180 (#179 #168)))) #170)))))) (λ#181. #169)) (λ#182. #182))))
3: (λ#183. (λ#184. (((λ#185. ((λ#186. (λ#187. (#187 (#186 #183)))) ((λ#188. (λ#189. (#189 (#188 #183)))) ((λ#190. (λ#191. (#191 (#190 #183)))) ((λ#192. (λ#193. (#193 (#192 #183)))) (λ#194. (#194 (#185 #183)))))))) (λ#195. #184)) (λ#196. #196))))
4: (λ#197. (λ#198. (((λ#199. ((λ#200. (λ#201. (#201 (#200 #197)))) (λ#202. (#202 (((λ#203. (λ#204. (#204 (#203 #197)))) ((λ#205. (λ#206. (#206 (#205 #197)))) (λ#207. (#207 (#19

# <span style="color: #A1F99C">6.</span>
<span style="color:#bb91f3">(PLUS 0 0)</span> == <span style="color:#f1bd47">0</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [9]:
def term_test_6():
    return multi_app_term(plus_term(), num_term(0), num_term(0))


test_procedure(term_test_6())

0: (((λ#296. (λ#297. ((#297 (λ#298. (λ#299. (λ#300. (#299 ((#298 #299) #300)))))) #296))) (λ#301. (λ#302. #302))) (λ#303. (λ#304. #304)))
1: ((λ#305. ((#305 (λ#306. (λ#307. (λ#308. (#307 ((#306 #307) #308)))))) (λ#309. (λ#310. #310)))) (λ#311. (λ#312. #312)))
2: (((λ#313. (λ#314. #314)) (λ#315. (λ#316. (λ#317. (#316 ((#315 #316) #317)))))) (λ#318. (λ#319. #319)))
3: ((λ#320. #320) (λ#321. (λ#322. #322)))
4: (λ#323. (λ#324. #324))


reduction steps: 4
norm term:       (λ#323. (λ#324. #324))


# <span style="color: #A1F99C">7.</span>
<span style="color:#bb91f3">(PLUS 2 3)</span> == <span style="color:#f1bd47">5</span> == <span style="color:#f1bd47">λx. λy. (x (x (x (x (x y)))))</span>

In [10]:
def term_test_7():
    return multi_app_term(plus_term(), num_term(2), num_term(3))


test_procedure(term_test_7())

0: (((λ#334. (λ#335. ((#335 (λ#336. (λ#337. (λ#338. (#337 ((#336 #337) #338)))))) #334))) (λ#339. (λ#340. (#339 (#339 #340))))) (λ#341. (λ#342. (#341 (#341 (#341 #342))))))
1: ((λ#343. ((#343 (λ#344. (λ#345. (λ#346. (#345 ((#344 #345) #346)))))) (λ#347. (λ#348. (#347 (#347 #348)))))) (λ#349. (λ#350. (#349 (#349 (#349 #350))))))
2: (((λ#351. (λ#352. (#351 (#351 (#351 #352))))) (λ#353. (λ#354. (λ#355. (#354 ((#353 #354) #355)))))) (λ#356. (λ#357. (#356 (#356 #357)))))
3: ((λ#358. ((λ#359. (λ#360. (λ#361. (#360 ((#359 #360) #361))))) ((λ#362. (λ#363. (λ#364. (#363 ((#362 #363) #364))))) ((λ#365. (λ#366. (λ#367. (#366 ((#365 #366) #367))))) #358)))) (λ#368. (λ#369. (#368 (#368 #369)))))
4: ((λ#370. ((λ#371. (λ#372. (λ#373. (#372 ((#371 #372) #373))))) ((λ#374. (λ#375. (λ#376. (#375 ((#374 #375) #376))))) (λ#377. (λ#378. (#377 ((#370 #377) #378))))))) (λ#379. (λ#380. (#379 (#379 #380)))))
5: ((λ#381. (λ#382. (λ#383. (#382 ((((λ#384. (λ#385. (λ#386. (#385 ((#384 #385) #386))))) (λ#387. (λ#38

# <span style="color: #A1F99C">8.</span>
<span style="color:#bb91f3">(SUBTRACT 4 1)</span> == <span style="color:#f1bd47">3</span> == <span style="color:#f1bd47">λx. λy. (x (x (x y)))</span>

In [11]:
def term_test_8():
    return multi_app_term(subtract_term(), num_term(4), num_term(1))


test_procedure(term_test_8())

0: (((λ#447. (λ#448. ((#448 (λ#449. (λ#450. (λ#451. (((#449 (λ#452. (λ#453. (#453 (#452 #450))))) (λ#454. #451)) (λ#455. #455)))))) #447))) (λ#456. (λ#457. (#456 (#456 (#456 (#456 #457))))))) (λ#458. (λ#459. (#458 #459))))
1: ((λ#460. ((#460 (λ#461. (λ#462. (λ#463. (((#461 (λ#464. (λ#465. (#465 (#464 #462))))) (λ#466. #463)) (λ#467. #467)))))) (λ#468. (λ#469. (#468 (#468 (#468 (#468 #469)))))))) (λ#470. (λ#471. (#470 #471))))
2: (((λ#472. (λ#473. (#472 #473))) (λ#474. (λ#475. (λ#476. (((#474 (λ#477. (λ#478. (#478 (#477 #475))))) (λ#479. #476)) (λ#480. #480)))))) (λ#481. (λ#482. (#481 (#481 (#481 (#481 #482)))))))
3: ((λ#483. ((λ#484. (λ#485. (λ#486. (((#484 (λ#487. (λ#488. (#488 (#487 #485))))) (λ#489. #486)) (λ#490. #490))))) #483)) (λ#491. (λ#492. (#491 (#491 (#491 (#491 #492)))))))
4: ((λ#493. (λ#494. (λ#495. (((#493 (λ#496. (λ#497. (#497 (#496 #494))))) (λ#498. #495)) (λ#499. #499))))) (λ#500. (λ#501. (#500 (#500 (#500 (#500 #501)))))))
5: (λ#502. (λ#503. ((((λ#504. (λ#505. (#504 (

# <span style="color: #A1F99C">9.</span>
<span style="color:#bb91f3">(SUBTRACT 3 5)</span> == <span style="color:#f1bd47">0</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [12]:
def term_test_9():
    return multi_app_term(subtract_term(), num_term(3), num_term(5))


test_procedure(term_test_9())

0: (((λ#612. (λ#613. ((#613 (λ#614. (λ#615. (λ#616. (((#614 (λ#617. (λ#618. (#618 (#617 #615))))) (λ#619. #616)) (λ#620. #620)))))) #612))) (λ#621. (λ#622. (#621 (#621 (#621 #622)))))) (λ#623. (λ#624. (#623 (#623 (#623 (#623 (#623 #624))))))))
1: ((λ#625. ((#625 (λ#626. (λ#627. (λ#628. (((#626 (λ#629. (λ#630. (#630 (#629 #627))))) (λ#631. #628)) (λ#632. #632)))))) (λ#633. (λ#634. (#633 (#633 (#633 #634))))))) (λ#635. (λ#636. (#635 (#635 (#635 (#635 (#635 #636))))))))
2: (((λ#637. (λ#638. (#637 (#637 (#637 (#637 (#637 #638))))))) (λ#639. (λ#640. (λ#641. (((#639 (λ#642. (λ#643. (#643 (#642 #640))))) (λ#644. #641)) (λ#645. #645)))))) (λ#646. (λ#647. (#646 (#646 (#646 #647))))))
3: ((λ#648. ((λ#649. (λ#650. (λ#651. (((#649 (λ#652. (λ#653. (#653 (#652 #650))))) (λ#654. #651)) (λ#655. #655))))) ((λ#656. (λ#657. (λ#658. (((#656 (λ#659. (λ#660. (#660 (#659 #657))))) (λ#661. #658)) (λ#662. #662))))) ((λ#663. (λ#664. (λ#665. (((#663 (λ#666. (λ#667. (#667 (#666 #664))))) (λ#668. #665)) (λ#669. #6

# <span style="color: #A1F99C">10.</span>
<span style="color:#bb91f3">(LEQ 3 2)</span> == <span style="color:#f1bd47">FALSE</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [13]:
def term_test_10():
    return multi_app_term(leq_term(), num_term(3), num_term(2))


test_procedure(term_test_10())

0: (((λ#1633. (λ#1634. ((λ#1635. ((#1635 (λ#1636. (λ#1637. (λ#1638. #1638)))) (λ#1639. (λ#1640. #1639)))) (((λ#1641. (λ#1642. ((#1642 (λ#1643. (λ#1644. (λ#1645. (((#1643 (λ#1646. (λ#1647. (#1647 (#1646 #1644))))) (λ#1648. #1645)) (λ#1649. #1649)))))) #1641))) #1633) #1634)))) (λ#1650. (λ#1651. (#1650 (#1650 (#1650 #1651)))))) (λ#1652. (λ#1653. (#1652 (#1652 #1653)))))
1: ((λ#1654. ((λ#1655. ((#1655 (λ#1656. (λ#1657. (λ#1658. #1658)))) (λ#1659. (λ#1660. #1659)))) (((λ#1661. (λ#1662. ((#1662 (λ#1663. (λ#1664. (λ#1665. (((#1663 (λ#1666. (λ#1667. (#1667 (#1666 #1664))))) (λ#1668. #1665)) (λ#1669. #1669)))))) #1661))) (λ#1670. (λ#1671. (#1670 (#1670 (#1670 #1671)))))) #1654))) (λ#1672. (λ#1673. (#1672 (#1672 #1673)))))
2: ((λ#1674. ((λ#1675. ((#1675 (λ#1676. (λ#1677. (λ#1678. #1678)))) (λ#1679. (λ#1680. #1679)))) ((λ#1681. ((#1681 (λ#1682. (λ#1683. (λ#1684. (((#1682 (λ#1685. (λ#1686. (#1686 (#1685 #1683))))) (λ#1687. #1684)) (λ#1688. #1688)))))) (λ#1689. (λ#1690. (#1689 (#1689 (#1689 #1690)

# <span style="color: #A1F99C">11.</span>
<span style="color:#bb91f3">(LEQ 2 5)</span> == <span style="color:#f1bd47">TRUE</span> == <span style="color:#f1bd47">λx. λy. x</span>

In [14]:
def term_test_11():
    return multi_app_term(leq_term(), num_term(2), num_term(5))


test_procedure(term_test_11())

0: (((λ#2161. (λ#2162. ((λ#2163. ((#2163 (λ#2164. (λ#2165. (λ#2166. #2166)))) (λ#2167. (λ#2168. #2167)))) (((λ#2169. (λ#2170. ((#2170 (λ#2171. (λ#2172. (λ#2173. (((#2171 (λ#2174. (λ#2175. (#2175 (#2174 #2172))))) (λ#2176. #2173)) (λ#2177. #2177)))))) #2169))) #2161) #2162)))) (λ#2178. (λ#2179. (#2178 (#2178 #2179))))) (λ#2180. (λ#2181. (#2180 (#2180 (#2180 (#2180 (#2180 #2181))))))))
1: (((λ#2182. (λ#2183. (((((λ#2184. (λ#2185. ((#2185 (λ#2186. (λ#2187. (λ#2188. (((#2186 (λ#2189. (λ#2190. (#2190 (#2189 #2187))))) (λ#2191. #2188)) (λ#2192. #2192)))))) #2184))) #2182) #2183) (λ#2193. (λ#2194. (λ#2195. #2195)))) (λ#2196. (λ#2197. #2196))))) (λ#2198. (λ#2199. (#2198 (#2198 #2199))))) (λ#2200. (λ#2201. (#2200 (#2200 (#2200 (#2200 (#2200 #2201))))))))
2: (((λ#2202. (λ#2203. ((((λ#2204. ((#2204 (λ#2205. (λ#2206. (λ#2207. (((#2205 (λ#2208. (λ#2209. (#2209 (#2208 #2206))))) (λ#2210. #2207)) (λ#2211. #2211)))))) #2202)) #2203) (λ#2212. (λ#2213. (λ#2214. #2214)))) (λ#2215. (λ#2216. #2215))))) (λ#

# <span style="color: #A1F99C">12.</span>
<span style="color:#bb91f3">(EQ 2 2)</span> == <span style="color:#f1bd47">TRUE</span> == <span style="color:#f1bd47">λx. λy. x</span>

In [15]:
def term_test_12():
    return multi_app_term(eq_term(), num_term(2), num_term(2))


test_procedure(term_test_12(), steps_lim=100)

0: (((λ#3191. (λ#3192. (((λ#3193. (λ#3194. ((((λ#3195. (λ#3196. (λ#3197. ((#3195 #3196) #3197)))) #3193) #3194) #3193))) (((λ#3198. (λ#3199. ((λ#3200. ((#3200 (λ#3201. (λ#3202. (λ#3203. #3203)))) (λ#3204. (λ#3205. #3204)))) (((λ#3206. (λ#3207. ((#3207 (λ#3208. (λ#3209. (λ#3210. (((#3208 (λ#3211. (λ#3212. (#3212 (#3211 #3209))))) (λ#3213. #3210)) (λ#3214. #3214)))))) #3206))) #3198) #3199)))) #3191) #3192)) (((λ#3215. (λ#3216. ((λ#3217. ((#3217 (λ#3218. (λ#3219. (λ#3220. #3220)))) (λ#3221. (λ#3222. #3221)))) (((λ#3223. (λ#3224. ((#3224 (λ#3225. (λ#3226. (λ#3227. (((#3225 (λ#3228. (λ#3229. (#3229 (#3228 #3226))))) (λ#3230. #3227)) (λ#3231. #3231)))))) #3223))) #3215) #3216)))) #3192) #3191)))) (λ#3232. (λ#3233. (#3232 (#3232 #3233))))) (λ#3234. (λ#3235. (#3234 (#3234 #3235)))))
1: (((λ#3236. (λ#3237. (((λ#3238. (λ#3239. (((λ#3240. (λ#3241. ((#3238 #3240) #3241))) #3239) #3238))) (((λ#3242. (λ#3243. ((λ#3244. ((#3244 (λ#3245. (λ#3246. (λ#3247. #3247)))) (λ#3248. (λ#3249. #3248)))) (((λ#32

# <span style="color: #A1F99C">13.</span>
<span style="color:#bb91f3">(EQ 1 4)</span> == <span style="color:#f1bd47">FALSE</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [16]:
def term_test_13():
    return multi_app_term(eq_term(), num_term(1), num_term(4))


test_procedure(term_test_13(), steps_lim=100)

0: (((λ#6431. (λ#6432. (((λ#6433. (λ#6434. ((((λ#6435. (λ#6436. (λ#6437. ((#6435 #6436) #6437)))) #6433) #6434) #6433))) (((λ#6438. (λ#6439. ((λ#6440. ((#6440 (λ#6441. (λ#6442. (λ#6443. #6443)))) (λ#6444. (λ#6445. #6444)))) (((λ#6446. (λ#6447. ((#6447 (λ#6448. (λ#6449. (λ#6450. (((#6448 (λ#6451. (λ#6452. (#6452 (#6451 #6449))))) (λ#6453. #6450)) (λ#6454. #6454)))))) #6446))) #6438) #6439)))) #6431) #6432)) (((λ#6455. (λ#6456. ((λ#6457. ((#6457 (λ#6458. (λ#6459. (λ#6460. #6460)))) (λ#6461. (λ#6462. #6461)))) (((λ#6463. (λ#6464. ((#6464 (λ#6465. (λ#6466. (λ#6467. (((#6465 (λ#6468. (λ#6469. (#6469 (#6468 #6466))))) (λ#6470. #6467)) (λ#6471. #6471)))))) #6463))) #6455) #6456)))) #6432) #6431)))) (λ#6472. (λ#6473. (#6472 #6473)))) (λ#6474. (λ#6475. (#6474 (#6474 (#6474 (#6474 #6475)))))))
1: (((λ#6476. (λ#6477. (((λ#6478. (λ#6479. ((((λ#6480. (λ#6481. (λ#6482. ((#6480 #6481) #6482)))) #6478) #6479) #6478))) (((λ#6483. (λ#6484. (((((λ#6485. (λ#6486. ((#6486 (λ#6487. (λ#6488. (λ#6489. (((#648